# Assignment 6
*This is an **individual** assignment.*

The objective of this assignment is to give you experience importing and using the [Pandas library](https://pandas.pydata.org/) for data manipulation.

Please meet the specific requirements exactly. Do not change or remove any of the instructions; just add you name and NetID in the Markdown cell below and code in two code cells as indicated.  Please make sure to test your programs before submitting your solutions on Canvas!  You can add extra cells to debug, but please remove them before submitting as it makes grading easier.

**Name:** Tirth Gala

**NetID:** tg260

## Part 1: Customer Segmentation

In this problem, you'll group customers by their demographic attributes to segment customers and compute the avaerage spending beavior by group.  This can help target marketing campaigns effectively.

### Step 1: Use `pandas` to load the data and take a look at how it is structured 

There are two data files: `customers.csv` and `purchases.csv`. 

In [1]:
# TODO: Load customer data from 'customers.csv'
import pandas as pd
customer_data = pd.read_csv("customers.csv")

In [2]:
# TODO: Load purchase data from 'purchases.csv'
purchase_data = pd.read_csv("purchases.csv")

### Step 2: Manipulate the data into more usable formats

There are a few issues with this data that we need to correct before using it. 

First: it's hard to group people based on indiivudal ages, so let's create three age buckets.  If you named your customer `DataFrame` `customer_data`, you can just use the cell below; otherwise you'll just need to change the `DataFrame` variable name to match what you did above.

In [3]:
# Add a new age group column so we can use that to group customers
bins= [18,35,55,100]
labels = ['18-34', '35-54', '55+']
customer_data['age.group'] = pd.cut(customer_data['age'], bins=bins, labels=labels, right=False)

Now notice the shape of the purchase data?  This is sometimes called "wide" data (but the term is not very precise).  It will be more useful to us in a three-column format with `customer.id`, `purchase.category`, and `spending` being the column names.  Use the `melt` method to achieve this.

In [4]:
# TODO: Melt the purchase data into a three-column data frame (customer.id, purchase.category, spending)

purchase_melt= pd.melt(purchase_data, id_vars=["customer.id"], value_vars=["Auto","Clothing & Accessories","Electronics","Furniture","Grocery","Housewares","Patio & Garden","Personal Care","Pharmacy & Health","Sports & Outdoors","Toys & Games"], var_name="purchase.category", value_name="spending")

### Step 3: Merge the data

Now you want to merge both the customer data and the purchase data into one `DataFrame`.

In [5]:
# TODO: merge customer and purchase data
m_purchase_data = pd.merge(customer_data, purchase_data, on='customer.id')

### Step 4: Compute and write out some useful summary data

Finally, we want to group the data by `age.group`, `gender`, and `purchase.category`; calculate average purchase amount per group; and write the resulting `DataFrame` out to `average_spending_by_demographics.csv`.  The first five lines of this file should look as follows.

```
age.group,gender,purchase.category,spending                                     
18-34,female,Auto,100.63157894736842                                            
18-34,female,Clothing & Accessories,98.42105263157895                           
18-34,female,Electronics,101.94736842105263                                     
18-34,female,Furniture,98.63157894736842
```

In [6]:
# TODO: Group data by age.group, gender, and purchase.category; calculate average purchase amount; and
# export the resulting DataFrame to average_spending_by_demographics.csv
merge_purchase_data = pd.merge(customer_data, purchase_melt, on='customer.id')
merge_purchase_data
grouping = merge_purchase_data.groupby(["age.group", "gender", "purchase.category"], axis=0).mean(numeric_only = True)
grouping.drop(["age","customer.id"],axis=1, inplace=True)
grouping.to_csv("average_spending_by_demographics.csv")

## Part 2: Sales Analysis

In this problem, you'll analyze sales data from different regions to identify top-selling products, brands, and regions.

### Step 1: Load the data and take a look at how it is structured! 

There are three data files: `sales.csv`, `regions.csv`, and `products.csv`.  Some of these files are comma-separated (`,`) and some use semicolons (`;`) for separating columns.  Why?  Some column text *contains* commas that we want to preserve, so using commas as separators wouldget ugly!

In [7]:
# TODO: Load the sales data from 'sales.csv'
sales_data = pd.read_csv('sales.csv',sep=';')

In [8]:
# TODO: Load region data from 'regions.csv'
regions_data = pd.read_csv("regions.csv")

In [9]:
# TODO: Load product data from 'products.csv'
products_data = pd.read_csv('products.csv',sep=';')

Notice the price column in the products data?  The prices all have dollar signs, which means that column contains strings rather than floats.  Let's get rid of it and convert the prices to floats!  If you named your products `DataFrame` `products`, you can just use the cell below; otherwise you'll just need to change the `DataFrame` variable name to match what you did above.

(P.S. we'll learn about `regex` next session!)

In [10]:
# Remove that pesky dollar sign and convert prices to floats!
products_data.price = products_data.price.str.replace('$', '', regex=False).astype(float)

### Step 2: Merge data

Now we want to merge all this data together into a single `DataFrame`.

In [11]:
# TODO: Merge the sales data frame with the regions data frame.  
# Hint: Watch out for column names that don't match!
m_sales_data = pd.merge(sales_data, regions_data, left_on='region', right_on='region.id')

In [12]:
# TODO: Merge the DataFrame you just created with the products data frame.
# Hint: You'll want to merge on *multiple columns*!
m_final_data = pd.merge(m_sales_data,products_data,on=['product.id','product.title'])

### Step 3: Create a `sales` column

Right now, we have a column for the `quantity` of sales for a given product in a given region, and the `price` of that product.  Use those to create a **new** column called `sales` that is the gross sales amount (in dollars) for that product in the region specified.

**Hint:** *Yes, it's as simple as multiplying the columns.*

In [13]:
# TODO: Create a new 'sales' column based on 'quantity' and 'price'
m_final_data["sales"] = m_final_data["quantity"] * m_final_data["price"]

### Step 4: Compute and write out some useful summary data

Your management team has three questions:
- Q1: What are our top-selling brands (by dollar amount)?
- Q2: What are our top-selling items (by dollar amount)?
- Q3: What are our top-selling regions (by dollar amount)?


Your task is to group, aggregate, and sort the data to answer these questions, and write out three files.


The first five lines of `top_selling_brands.csv` should look like this.

```
brand,sales                                                                     
Lifetime,92843864.87                                                            
ECR4Kids,57093850.48                                                            
American Educational Products,56568976.25                                       
LEGO,43421577.93 
```

The first five lines of `top_selling_products.csv` should look like this.

```
product.id;product.title;sales                                                  
B004J2HY2M;Lifetime Monkey Bar Adventure Swing Set with 9 Foot Wavy Slide;56158438.410000004
B00C9G4V3K;EISCO Steel Goggle Sanitizer Cabinet, 120 VAC, For 35 Pairs;20561079.330000002
B004ZAITYK;ECR4Kids SoftZone Single Tunnel Maze Foam Play Climber;19423168.75   
B00NBHXW9U;WeDo Construction Set;13647155.02 
```

The first five lines of `top_selling_regions.csv` should look like this.

```
region,sales                                                                    
26,55334107.13                                                                  
17,53234412.76                                                                  
25,49942514.77                                                                  
56,44038923.27
```

*Note this assignment will be evaluated on your ability to write code to generate the full `csv` files, not to actually answer the questions.  In the real world, answering the questions is obviously more important, but right now we're focused on sharpening your programming and data managemnt skills!*

In [14]:
# TODO: To answer Q1, group the data by brand and sum the sales column. Then sort 
# the rows by sales so the largest is at the top.  Finally, write out the result to
# 'top_selling_brands.csv` so it matches the example lines above.
top_selling_brands = m_final_data.groupby('brand')['sales'].sum(numeric_only=True).reset_index()
top_selling_brands = top_selling_brands.sort_values(by='sales', ascending=False)
top_selling_brands.to_csv("top_selling_brands.csv", index=False,header=True)

In [15]:
# TODO: To answer Q2, group the data by product and sum the sales column. Then sort 
# the rows by sales so the largest is at the top.  Finally, write out the result to
# 'top_selling_products.csv` so it matches the example lines above.
# Hint: You may need to specifiy more than one column to group by!
top_selling_products = m_final_data.groupby(['product.id','product.title'])['sales'].sum(numeric_only=True).reset_index()
top_selling_products = top_selling_products.sort_values(by='sales', ascending=False)
top_selling_products.to_csv("top_selling_products.csv", sep=";", index=False, header=True)

In [16]:
# TODO: To answer Q3, group the data by product and sum the sales column. Then sort 
# the rows by sales so the largest is at the top.  Finally, write out the result to
# 'top_selling_products.csv` so it matches the example lines above.
# Hint: You may need to specifiy more than one column to group by!
top_selling_regions = m_final_data.groupby('region')['sales'].sum(numeric_only=True).reset_index()
top_selling_regions = top_selling_regions.sort_values(by='sales', ascending=False)
top_selling_regions.to_csv("top_selling_regions.csv",index=False,header=True)